# Multi-Modal Financial Analysis Agent: Final Submission

**Course:** AAI 520 - Final Project

This notebook implements the final version of a multi-modal AI system that performs comparative analysis on multiple stocks using market, macroeconomic, and news sentiment data.

## Setup and Dependencies
This cell installs the required `vaderSentiment` library, imports all necessary packages, and sets the API keys.**bold text**

In [1]:
'''Uncomment and run the following line if you haven't installed the required packages yet'''
#!py -m pip install openai python-dotenv yfinance pydantic requests vaderSentiment google-generativeai tabulate seaborn matplotlib

"Uncomment and run the following line if you haven't installed the required packages yet"

In [2]:
import os, shutil
import pandas as pd
import yfinance as yf
import requests
from IPython.display import display, Markdown
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
from __future__ import annotations
import os, json, time, argparse, datetime as dt
import openai
import hashlib, hmac, math
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Callable, Iterable
import math
from dataclasses import dataclass, field
from pydantic import BaseModel
from pandas.api.types import is_datetime64_any_dtype
import textwrap
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
import base64
from IPython.display import display, HTML

from dotenv import load_dotenv

# tqdm (nice progress bars)
try:
    from tqdm.auto import tqdm
    _HAS_TQDM = True
except Exception:
    _HAS_TQDM = False

### Configuration

Enviroment set with .env file as follows:

```env
# Economic data from the Federal Reserve
FRED_API_KEY="YOUR_FRED_API_KEY"

# Stock market and financial data
POLYGON_API_KEY="YOUR_POLYGON_API_KEY"
FINNHUB_API_KEY="YOUR_FINNHUB_API_KEY"

# Real-time news articles
NEWS_API_KEY="YOUR_NEWS_API_KEY"

# For the AI agent's "brain"
OPENAI_API_KEY="YOUR_OPENAI_API_KEY"
GOOGLE_API_KEY="YOUR_GOOGLE_API_KEY"

# Identification for accessing SEC's EDGAR database
SEC_USER_AGENT="Your Name you@example.com"
```

In [3]:
# Load environment variables from .env file
load_dotenv()

# Set API keys from environment variables
FRED_KEY        = os.getenv("FRED_API_KEY")
NEWS_KEY        = os.getenv("NEWS_API_KEY")
FINNHUB_KEY     = os.getenv("FINNHUB_API_KEY")
POLYGON_KEY     = os.getenv("POLYGON_API_KEY")
OPENAI_KEY      = os.getenv("OPENAI_API_KEY")
GEMINI_KEY      = os.getenv("GOOGLE_API_KEY")
SEC_USER_AGENT  = os.getenv("SEC_USER_AGENT")

# Configuration
OPENAI_MODEL = "gpt-4o-mini"    
GEMINI_MODEL = "gemini-2.5-flash"    
NEWS_STORE   = "news_store.parquet" 
CACHE_DIR    = ".cache"             

# Ensure cache directory exists
Path(CACHE_DIR).mkdir(exist_ok=True)

### Test LLM Models

In [4]:
import google.generativeai as genai
from openai import OpenAI

def check_llm_availability():
    """
    Checks the availability and functionality of configured LLM APIs.
    This function is designed to be run directly in a notebook cell.
    """
        
    print("--- Checking LLM Availability ---")
    # --- Test 1: Google Gemini ---
    try:
        assert GEMINI_KEY, "GOOGLE_API_KEY is not set in your environment."
        genai.configure(api_key=GEMINI_KEY)
        
        # Using a reliable and recent model
        model = genai.GenerativeModel("gemini-2.5-flash")
        prompt = 'Return ONLY this JSON: {"ok": true}'
        response = model.generate_content(prompt, generation_config={"temperature": 0})
        if "ok" in response.text:
            print("✅ Google Gemini: OK")
        else:
            print(f"❌ Google Gemini: Unexpected response -> {response.text.strip()}")
            
    except Exception as e:
        print(f"❌ Google Gemini: FAILED - {e}")

    # --- Test 2: OpenAI GPT ---
    try:
        assert OPENAI_KEY, "OPENAI_API_KEY is not set in your environment."
        client = OpenAI(api_key=OPENAI_KEY)
        response = client.chat.completions.create(
            model = OPENAI_MODEL,
            messages=[{"role": "user", "content": "Reply with exactly: OK"}],
            temperature=0
        )
        
        output_text = response.choices[0].message.content.strip()
        if output_text == "OK":
            print("✅ OpenAI GPT: OK")
        else:
            print(f"❌ OpenAI GPT: Unexpected response -> {output_text}")

    except Exception as e:
        print(f"❌ OpenAI GPT: FAILED - {e}")

# Check now
check_llm_availability()

--- Checking LLM Availability ---
✅ Google Gemini: OK
✅ OpenAI GPT: OK


## Data Tools and Helper Classes/Functions

Lightweight Utils (DiskCache + stable id)

In [5]:
# -------------------------------
# Persistent Memory (tiny JSON)
# -------------------------------
class MemoryStore:
    """
    A simple JSON-based memory store for the agent to learn across runs.
    Saves and retrieves brief notes about symbols.
    """
    def __init__(self, path: str = ".agent_memory.json"):
        self.path = path
        if not os.path.exists(self.path):
            with open(self.path, "w", encoding="utf-8") as f:
                json.dump({"symbols": {}}, f)

    def _load(self) -> Dict[str, Any]:
        with open(self.path, "r", encoding="utf-8") as f:
            return json.load(f)

    def _save(self, data: Dict[str, Any]):
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2)

    def add_note(self, symbol: str, note: str):
        """Adds a new memory note for a given stock symbol."""
        data = self._load()
        symbols = data.setdefault("symbols", {})
        note_list = symbols.setdefault(symbol.upper(), [])
        
        timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
        note_list.append({"ts": timestamp, "note": note})
        self._save(data)
        print(f"   - Memory added for {symbol.upper()}")

    def get_notes(self, symbol: str, last_n: int = 5) -> str:
        """Retrieves the last N notes for a symbol as a single string."""
        data = self._load()
        notes = data.get("symbols", {}).get(symbol.upper(), [])
        if not notes:
            return "No past notes found for this symbol."
        
        formatted_notes = [f"- {n['ts']}: {n['note']}" for n in notes[-last_n:]]
        return "### Past Analysis Notes:\n" + "\n".join(formatted_notes)
    
# -------------------------------
# Disk Cache (parquet files)
# -------------------------------
class DiskCache:
    # ... (This class is correct, no changes needed) ...
    def __init__(self, cache_dir: str, ttl_seconds: int):
        self.cache_dir = cache_dir
        self.ttl_seconds = ttl_seconds
        os.makedirs(self.cache_dir, exist_ok=True)
    def _cache_path(self, key: str) -> str:
        h = hashlib.sha1(key.encode("utf-8")).hexdigest()
        return os.path.join(self.cache_dir, f"{h}.parquet")
    def get(self, key: str) -> pd.DataFrame | None:
        path = self._cache_path(key)
        if not os.path.exists(path): return None
        if (time.time() - os.path.getmtime(path)) > self.ttl_seconds: return None
        try: return pd.read_parquet(path)
        except Exception: return None
    def set(self, key: str, df: pd.DataFrame):
        path = self._cache_path(key)
        df.to_parquet(path, index=False)

### Economic Data From FRED

In [6]:
class EconomicDataTool:
    """
    A tool to fetch economic data series from the FRED API.
    """
    BASE_URL = "https://api.stlouisfed.org/fred/series/observations"

    def __init__(self, cache_dir: str = ".cache/fred", ttl_seconds: int = 12 * 3600):
        self.api_key = os.getenv("FRED_API_KEY")
        if not self.api_key:
            print("⚠️ FRED_API_KEY not set. The EconomicDataTool will be disabled.")
        
        self.cache_dir = cache_dir
        self.ttl_seconds = ttl_seconds
        os.makedirs(self.cache_dir, exist_ok=True)

    def _cache_path(self, key: str) -> str:
        h = hashlib.sha1(key.encode("utf-8")).hexdigest()
        return os.path.join(self.cache_dir, f"{h}.parquet")

    def _read_cache(self, key: str) -> pd.DataFrame | None:
        path = self._cache_path(key)
        if not os.path.exists(path): return None
        if (time.time() - os.path.getmtime(path)) > self.ttl_seconds: return None
        try: return pd.read_parquet(path)
        except Exception: return None

    def _write_cache(self, key: str, df: pd.DataFrame):
        path = self._cache_path(key)
        df.to_parquet(path, index=False)

    def get_series(self, series_ids: list[str], start_date: str = "2020-01-01") -> pd.DataFrame:
        """
        Fetches one or more economic data series from FRED and merges them.
        
        Common Series IDs:
        - GDP: Real Gross Domestic Product
        - CPIAUCSL: Consumer Price Index (Inflation)
        - UNRATE: Unemployment Rate
        - FEDFUNDS: Federal Funds Effective Rate
        """
        if not self.api_key:
            return pd.DataFrame()

        # Create a stable cache key from the sorted list of series
        sorted_ids = sorted(series_ids)
        cache_key = f"fred::{'&'.join(sorted_ids)}::{start_date}"
        
        cached_df = self._read_cache(cache_key)
        if cached_df is not None:
            return cached_df

        all_series_dfs = []
        for series_id in sorted_ids:
            params = {
                "series_id": series_id,
                "api_key": self.api_key,
                "file_type": "json",
                "observation_start": start_date,
            }
            try:
                response = requests.get(self.BASE_URL, params=params, timeout=30)
                response.raise_for_status()
                data = response.json().get("observations", [])
                
                if not data:
                    print(f"No data returned for FRED series: {series_id}")
                    continue

                df = pd.DataFrame(data)
                df = df[["date", "value"]]
                df = df.rename(columns={"value": series_id})
                
                # Clean the data
                df["date"] = pd.to_datetime(df["date"])
                # FRED uses '.' for missing values
                df[series_id] = pd.to_numeric(df[series_id], errors='coerce')
                
                all_series_dfs.append(df)
            except requests.exceptions.RequestException as e:
                print(f"Failed to fetch FRED series {series_id}: {e}")
        
        if not all_series_dfs:
            return pd.DataFrame()

        # Merge all individual series DataFrames into one
        merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), all_series_dfs)
        merged_df = merged_df.sort_values('date', ascending=False).reset_index(drop=True)
        
        self._write_cache(cache_key, merged_df)
        return merged_df

### Test

In [7]:
def run_economic_data_tool_smoke_test():
    """
    A simple test to verify the EconomicDataTool is working correctly.
    """
    print("--- 💨 Running Smoke Test for EconomicDataTool ---")
    
    # Ensure environment variables are loaded (especially FRED_API_KEY)
    load_dotenv()
    
    # 1. Instantiate the tool
    tool = EconomicDataTool()
    
    # 2. Check if the API key is available before proceeding
    if not tool.api_key:
        print("❌ Test SKIPPED: FRED_API_KEY is not set in your environment.")
        return

    # 3. Define a few common and reliable FRED series IDs to fetch
    series_to_fetch = {
        "GDP": "Real Gross Domestic Product",
        "CPIAUCSL": "Consumer Price Index (Inflation)",
        "UNRATE": "Unemployment Rate"
    }
    
    print(f"Fetching series: {', '.join(series_to_fetch.keys())}...")
    
    # 4. Call the tool's main method
    df = tool.get_series(series_ids=list(series_to_fetch.keys()))
    
    # 5. Verify the output
    if df is not None and not df.empty:
        print(f"\n✅ Test PASSED: Successfully fetched {len(df)} data points.")
        print("--- Sample of Fetched Economic Data ---")
        display(df.head())
    else:
        print("\n❌ Test FAILED: The tool returned an empty DataFrame.")
        print("   Please check your FRED_API_KEY and network connection.")

# --- Execute the smoke test ---
run_economic_data_tool_smoke_test()

--- 💨 Running Smoke Test for EconomicDataTool ---
Fetching series: GDP, CPIAUCSL, UNRATE...

✅ Test PASSED: Successfully fetched 68 data points.
--- Sample of Fetched Economic Data ---


,date,CPIAUCSL,GDP,UNRATE
0,2025-08-01,323.364,NaN,4.3
1,2025-07-01,322.132,NaN,4.2
2,2025-06-01,321.500,NaN,4.1
3,2025-05-01,320.580,NaN,4.2
4,2025-04-01,320.321,30485.729,4.2


### Market Data From Yahoo Finance 

In [8]:
class MarketDataTool:
    """
    Market data access + light feature engineering (optional).
    - Standardized schema: ['date','open','high','low','close','volume']
    - Intraday support (1m/2m/5m/15m/30m/60m/90m/1h)
    - Simple on-disk caching with TTL
    - Batch fetch for multiple tickers -> long format with a 'ticker' column
    """

    def __init__(
        self,
        cache_dir: str = ".cache/yfinance",
        ttl_seconds: int = 3600,
        max_retries: int = 2,
        pause_between_retries: float = 0.7
    ):
        self.cache_dir = cache_dir
        self.ttl_seconds = ttl_seconds
        self.max_retries = max_retries
        self.pause_between_retries = pause_between_retries
        os.makedirs(self.cache_dir, exist_ok=True)

    # ---------------------------
    # Core helpers
    # ---------------------------
    def _cache_path(self, key: str) -> str:
        h = hashlib.sha1(key.encode("utf-8")).hexdigest()
        return os.path.join(self.cache_dir, f"{h}.parquet")

    def _read_cache(self, key: str) -> Optional[pd.DataFrame]:
        path = self._cache_path(key)
        if not os.path.exists(path):
            return None
        if (time.time() - os.path.getmtime(path)) > self.ttl_seconds:
            return None
        try:
            return pd.read_parquet(path)
        except Exception:
            # Fallback to CSV if parquet fails (rare)
            alt = path.replace(".parquet", ".csv")
            if os.path.exists(alt):
                try:
                    return pd.read_csv(alt, parse_dates=["date"])
                except Exception:
                    return None
            return None

    def _write_cache(self, key: str, df: pd.DataFrame) -> None:
        path = self._cache_path(key)
        try:
            df.to_parquet(path, index=False)
        except Exception:
            df.to_csv(path.replace(".parquet", ".csv"), index=False)

    def _normalize_columns(self, df: pd.DataFrame, ticker: str) -> pd.DataFrame:
        import pandas as pd
        from pandas.api.types import is_datetime64_any_dtype

        # Ensure a DataFrame (some paths may pass a Series or dict-like)
        df = pd.DataFrame(df).copy()

        # Reset index to surface the datetime index as a column (Date/Datetime/index)
        df = df.reset_index()

        # Normalize columns: flatten tuples, lowercase, underscores
        df.columns = [
            "_".join(str(s) for s in col if s) if isinstance(col, tuple) else str(col)
            for col in df.columns
        ]
        df.columns = [c.lower().replace(" ", "_") for c in df.columns]

        # --- Find/standardize the datetime column to 'date' ---
        # 1) Prefer a column already of datetime dtype
        dt_cols = [c for c in df.columns if is_datetime64_any_dtype(df[c])]
        date_col = dt_cols[0] if dt_cols else None

        # 2) Otherwise look for common names and parse
        if date_col is None:
            for cand in ("date", "datetime", "timestamp", "index"):
                if cand in df.columns:
                    # try to parse to datetime
                    df[cand] = pd.to_datetime(df[cand], errors="coerce", utc=False)
                    if is_datetime64_any_dtype(df[cand]):
                        date_col = cand
                        break

        # 3) If still missing, last resort: try to_datetime on the first column
        if date_col is None and len(df.columns) > 0:
            first = df.columns[0]
            df[first] = pd.to_datetime(df[first], errors="coerce", utc=False)
            if is_datetime64_any_dtype(df[first]):
                date_col = first

        if date_col is None:
            # Cannot reliably identify a datetime column; return empty with expected schema
            return pd.DataFrame(columns=["date", "open", "high", "low", "close", "volume"])

        if date_col != "date":
            df = df.rename(columns={date_col: "date"})

        # --- Map OHLCV names (handles multi-ticker suffixes like open_aapl) ---
        t = ticker.lower()
        colmap = {
            f"open_{t}": "open",
            f"high_{t}": "high",
            f"low_{t}": "low",
            f"close_{t}": "close",
            f"volume_{t}": "volume",
        }
        df = df.rename(columns=colmap)

        # Prefer adj_close if close missing
        if "adj_close" in df.columns and "close" not in df.columns:
            df = df.rename(columns={"adj_close": "close"})

        # Cast numeric safely
        for c in ("open", "high", "low", "close", "volume"):
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors="coerce")

        # Ensure datetime
        df["date"] = pd.to_datetime(df["date"], errors="coerce")

        # Drop bad rows
        df = df.dropna(subset=["date", "close"]).reset_index(drop=True)

        # Final schema (return empty with correct cols if missing)
        required = ["date", "open", "high", "low", "close", "volume"]
        missing = [c for c in required if c not in df.columns]
        if missing:
            # Create any missing required columns as NaN to keep schema stable
            for c in missing:
                df[c] = pd.NA
            df = df[required]

        return df[required]


    def _yf_download(self, tickers, **kwargs):
        """
        Thin wrapper with simple retries to handle intermittent YF hiccups.
        """
        err = None
        for attempt in range(self.max_retries + 1):
            try:
                return yf.download(tickers, progress=False, auto_adjust=True, **kwargs)
            except Exception as e:
                err = e
                time.sleep(self.pause_between_retries * (attempt + 1))
        raise err if err else RuntimeError("Unknown yfinance error")

    # ---------------------------
    # Public API
    # ---------------------------
    def get_stock_prices(
        self,
        ticker: str,
        period: str = "5y",
        interval: str = "1d"
    ) -> pd.DataFrame:
        """
        Single-ticker normalized OHLCV.
        Returns standardized columns: ['date','open','high','low','close','volume'].
        Caches results for ttl_seconds.
        """
        key = f"single::{ticker}::{period}::{interval}"
        cached = self._read_cache(key)
        if cached is not None:
            return cached

        # yfinance can return tuple in some environments; normalize robustly.
        try:
            result = self._yf_download(ticker, period=period, interval=interval)
        except Exception as e:
            print(f"Error fetching stock data for {ticker}: {e}")
            return pd.DataFrame(columns=["date","open","high","low","close","volume"])

        data = result[0] if isinstance(result, tuple) else result
        if data is None or data.empty:
            return pd.DataFrame(columns=["date","open","high","low","close","volume"])

        df = self._normalize_columns(data, ticker)
        self._write_cache(key, df)
        return df

    def batch_get_prices(
        self,
        tickers: List[str],
        period: str = "1y",
        interval: str = "1d"
    ) -> pd.DataFrame:
        """
        Multi-ticker fetch. Returns LONG format:
        ['ticker','date','open','high','low','close','volume'].
        Works whether yfinance returns a flat frame or a column MultiIndex.
        """
        # Cache key is content-addressed by sorted tickers for determinism
        tickers_sorted = sorted(set([t.upper() for t in tickers]))
        key = f"batch::{','.join(tickers_sorted)}::{period}::{interval}"
        cached = self._read_cache(key)
        if cached is not None:
            return cached

        try:
            result = self._yf_download(tickers_sorted, period=period, interval=interval)
        except Exception as e:
            print(f"Error fetching batch data: {e}")
            return pd.DataFrame(columns=["ticker","date","open","high","low","close","volume"])

        if result is None or result.empty:
            return pd.DataFrame(columns=["ticker","date","open","high","low","close","volume"])

        # yfinance for multiple tickers returns a wide MultiIndex columns like:
        # ('Open','AAPL'), ('High','AAPL'), ...
        # If single ticker slips through, handle as single
        if not isinstance(result.columns, pd.MultiIndex):
            # Single-like case; just normalize and add ticker
            # Try to guess which ticker it belongs to: use first of list
            base_ticker = tickers_sorted[0]
            df = self._normalize_columns(result, base_ticker)
            df.insert(0, "ticker", base_ticker)
            self._write_cache(key, df)
            return df

        # MultiIndex -> long
        out_frames = []
        # Top level should be ('Adj Close','Close','High','Low','Open','Volume')
        # Second level are tickers
        for t in tickers_sorted:
            sub = result.xs(t, axis=1, level=1, drop_level=False)
            # Rebuild a single-ticker frame with expected column names
            # Columns might be ('Open', t), etc.
            tmp = pd.DataFrame({
                "date": result.index
            })
            # Use get to be robust to missing columns
            def col2(s1): return (s1, t) if (s1, t) in sub.columns else None

            for src, dst in [("Open","open"),("High","high"),("Low","low"),("Close","close"),("Adj Close","adj_close"),("Volume","volume")]:
                c = col2(src)
                if c is not None:
                    tmp[dst] = sub[c].values

            tmp = self._normalize_columns(tmp, t)
            if tmp.empty:
                continue
            tmp.insert(0, "ticker", t)
            out_frames.append(tmp)

        if not out_frames:
            return pd.DataFrame(columns=["ticker","date","open","high","low","close","volume"])

        df_long = pd.concat(out_frames, ignore_index=True)
        self._write_cache(key, df_long)
        return df_long

    def get_price_panel(
        self,
        ticker: str,
        period: str = "6mo",
        interval: str = "1d",
        with_features: bool = True
    ) -> pd.DataFrame:
        """
        Convenience wrapper used by the agent's router.
        Adds light features if requested.
        """
        df = self.get_stock_prices(ticker, period=period, interval=interval)
        if df.empty or not with_features:
            return df
        df = df.copy()
        df["pct_change"] = df["close"].pct_change()
        df["ret_20d"] = df["close"] / df["close"].shift(20) - 1.0
        df["sma_20"] = df["close"].rolling(20, min_periods=5).mean()
        df["sma_50"] = df["close"].rolling(50, min_periods=10).mean()
        df["vol_ma_20"] = df["volume"].rolling(20, min_periods=5).mean()
        return df

### TESTING

In [9]:
## Test single ticker fetch
mdt = MarketDataTool(ttl_seconds=3600)

# Daily, 5 years
aapl = mdt.get_stock_prices("AAPL", period="5y", interval="1d")

# Intraday (e.g., 5-minute). If your period is too long for the interval,
# yfinance will just return what it can; the cache keeps it consistent across runs.
nvda_5m = mdt.get_stock_prices("NVDA", period="60d", interval="5m")

# Panel w/ features for router hints
panel = mdt.get_price_panel("MSFT", period="6mo", interval="1d", with_features=True)

# -------------------------------
display(aapl.tail())
display(nvda_5m.tail())   

,date,open,high,low,close,volume
1251,2025-10-13,249.380005,249.690002,245.559998,247.660004,38142900
1252,2025-10-14,246.600006,248.850006,244.699997,247.770004,35478000
1253,2025-10-15,249.490005,251.820007,247.470001,249.339996,33893600
1254,2025-10-16,248.250000,249.039993,245.130005,247.449997,39777000
1255,2025-10-17,248.020004,253.380005,247.270004,252.289993,48876500


,date,open,high,low,close,volume
4669,2025-10-17 19:35:00+00:00,183.380005,183.520004,183.184998,183.259995,1476782
4670,2025-10-17 19:40:00+00:00,183.279999,183.350006,183.160004,183.189804,1250507
4671,2025-10-17 19:45:00+00:00,183.184998,183.279999,183.029999,183.279999,1419579
4672,2025-10-17 19:50:00+00:00,183.259995,183.279007,182.839996,183.095001,2574806
4673,2025-10-17 19:55:00+00:00,183.100006,183.789993,182.985001,183.240005,3930255


## NewsTool

In [10]:
class NewsDataTool:
    """
    Company news access with robust normalization + TTL parquet cache.

    Standardized columns:
      ['symbol','source','publisher','published_utc','headline','summary','url']

    Behavior mirrors MarketDataTool:
      - On-disk caching (parquet) with TTL
      - Simple retries
      - Batch fetch across tickers -> long format with 'symbol' column
    """
    def __init__(
        self,
        cache_dir: str = ".cache/news",
        ttl_seconds: int = 20 * 60,      # short TTL — news changes quickly
        max_retries: int = 2,
        pause_between_retries: float = 0.7,
        finnhub_key: str | None = None,
        polygon_key: str | None = None,
    ):
        import os
        self.cache_dir = cache_dir
        self.ttl_seconds = ttl_seconds
        self.max_retries = max_retries
        self.pause_between_retries = pause_between_retries
        self.finnhub_key = finnhub_key or FINNHUB_KEY
        self.polygon_key = polygon_key or POLYGON_KEY
        os.makedirs(self.cache_dir, exist_ok=True)

    # ---------- schema ----------
    @staticmethod
    def columns() -> list[str]:
        return ["symbol","source","publisher","published_utc","headline","summary","url"]

    # ---------- cache helpers ----------
    def _cache_path(self, key: str) -> str:
        import os, hashlib
        h = hashlib.sha1(key.encode("utf-8")).hexdigest()
        return os.path.join(self.cache_dir, f"{h}.parquet")

    def _read_cache(self, key: str):
        import os, time, pandas as pd
        path = self._cache_path(key)
        if not os.path.exists(path):
            return None
        if (time.time() - os.path.getmtime(path)) > self.ttl_seconds:
            return None
        try:
            df = pd.read_parquet(path)
            # ensure datetime tz-aware
            if "published_utc" in df.columns:
                df["published_utc"] = pd.to_datetime(df["published_utc"], utc=True, errors="coerce")
            return df
        except Exception:
            return None

    def _write_cache(self, key: str, df):
        path = self._cache_path(key)
        try:
            df.to_parquet(path, index=False)
        except Exception:
            # last-resort CSV
            df.to_csv(path.replace(".parquet",".csv"), index=False)

    # ---------- utils ----------
    @staticmethod
    def _safe_fix_text(x) -> str:
        from ftfy import fix_text
        import json
        if x is None:
            return ""
        if isinstance(x, str):
            return fix_text(x)
        if isinstance(x, dict):
            for k in ("summary","content","description","title","text","value"):
                v = x.get(k)
                if isinstance(v, str):
                    return fix_text(v)
            try:
                return fix_text(json.dumps(x, ensure_ascii=False, separators=(",", ":")))
            except Exception:
                return fix_text(str(x))
        if isinstance(x, list):
            parts = []
            for e in x:
                if isinstance(e, str):
                    parts.append(e)
                elif isinstance(e, dict):
                    parts.append(NewsDataTool._safe_fix_text(e))
            return fix_text(" ".join(p for p in parts if p))
        return fix_text(str(x))

    def _retry_get(self, url: str, params: dict, timeout: int = 20):
        import requests, time
        err = None
        for attempt in range(self.max_retries + 1):
            try:
                r = requests.get(url, params=params, timeout=timeout)
                r.raise_for_status()
                return r
            except Exception as e:
                err = e
                time.sleep(self.pause_between_retries * (attempt + 1))
        print(f"HTTP error: {url} | {err}")
        return None

    # ---------- per-source fetchers ----------
    def _fetch_yahoo(self, sym: str, max_items: int):
        import pandas as pd, yfinance as yf
        t = yf.Ticker(sym)
        raw = t.news or []
        rows = []
        for row in raw[:max_items]:
            ts_epoch = row.get("providerPublishTime") or row.get("pubDate")
            ts = pd.to_datetime(ts_epoch, unit="s", utc=True, errors="coerce") if ts_epoch else pd.NaT

            pub = row.get("publisher")
            if not isinstance(pub, str):
                prov = row.get("provider")
                if isinstance(prov, dict):
                    pub = prov.get("displayName")
                elif isinstance(prov, list) and prov and isinstance(prov[0], dict):
                    pub = prov[0].get("displayName")
            if not isinstance(pub, str):
                pub = None

            rows.append({
                "symbol": sym.upper(),
                "source": "Yahoo",
                "publisher": pub,
                "published_utc": ts,
                "headline": self._safe_fix_text(row.get("title") or row.get("headline") or ""),
                "summary":  self._safe_fix_text(row.get("summary") or row.get("content") or row.get("description") or ""),
                "url": row.get("link") or row.get("url") or "",
            })
        return pd.DataFrame(rows, columns=self.columns())

    def _fetch_finnhub(self, sym: str, days: int, max_items: int):
        import pandas as pd, datetime as dt
        if not self.finnhub_key:
            return pd.DataFrame(columns=self.columns())
        to = dt.date.today(); fr = to - dt.timedelta(days=days)
        r = self._retry_get(
            "https://finnhub.io/api/v1/company-news",
            {"symbol": sym, "from": fr.isoformat(), "to": to.isoformat(), "token": self.finnhub_key}
        )
        data = [] if r is None else (r.json() or [])
        rows = []
        for row in data[:max_items]:
            rows.append({
                "symbol": sym.upper(),
                "source": "Finnhub",
                "publisher": row.get("source") or None,
                "published_utc": pd.to_datetime(row.get("datetime",0), unit="s", utc=True, errors="coerce"),
                "headline": self._safe_fix_text(row.get("headline") or row.get("title") or ""),
                "summary":  self._safe_fix_text(row.get("summary") or row.get("description") or row.get("text") or ""),
                "url": row.get("url") or "",
            })
        return pd.DataFrame(rows, columns=self.columns())

    def _fetch_polygon(self, sym: str, limit: int):
        import pandas as pd
        if not self.polygon_key:
            return pd.DataFrame(columns=self.columns())
        r = self._retry_get(
            "https://api.polygon.io/v2/reference/news",
            {"ticker": sym, "limit": min(limit, 1000), "apiKey": self.polygon_key}
        )
        data = [] if r is None else ((r.json() or {}).get("results", []) or [])
        rows = []
        for row in data:
            pub = row.get("publisher")
            if isinstance(pub, dict):
                pub = pub.get("name")
            rows.append({
                "symbol": sym.upper(),
                "source": "Polygon",
                "publisher": pub,
                "published_utc": pd.to_datetime(row.get("published_utc") or None, utc=True, errors="coerce"),
                "headline": self._safe_fix_text(row.get("title") or ""),
                "summary":  self._safe_fix_text(row.get("description") or row.get("summary") or ""),
                "url": row.get("article_url") or row.get("amp_url") or "",
            })
        return pd.DataFrame(rows, columns=self.columns())

    # ---------- orchestrators ----------
    def fetch_one(
        self,
        symbol: str,
        days: int = 7,
        max_per_source: int = 120,
        use_sources: list[str] | None = None,
        relevance_fn = None,  # optional: lambda sym, headline, summary -> bool
    ):
        """
        Single-symbol fetch with normalization, optional relevance filter,
        dedupe by URL, newest-first. Cached by (symbol, days, max_per_source, sources).
        """
        import pandas as pd, os
        symbol = symbol.upper()
        use_sources = [s.lower() for s in (use_sources or ["yahoo","finnhub","polygon"])]
        key = f"news::{symbol}::d{days}::m{max_per_source}::src{','.join(use_sources)}"
        cached = self._read_cache(key)
        if cached is not None:
            df = cached
        else:
            frames = []
            if "yahoo"   in use_sources: frames.append(self._fetch_yahoo(symbol, max_per_source))
            if "finnhub" in use_sources: frames.append(self._fetch_finnhub(symbol, days, max_per_source))
            if "polygon" in use_sources: frames.append(self._fetch_polygon(symbol, max_per_source))
            df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=self.columns())

            if not df.empty:
                df["published_utc"] = pd.to_datetime(df["published_utc"], utc=True, errors="coerce")
                df["url"] = df["url"].fillna("").astype(str)
                df = df.sort_values("published_utc", ascending=False).drop_duplicates(subset=["url"]).reset_index(drop=True)

            self._write_cache(key, df)

        if df.empty:
            return df

        # optional ticker relevance
        if relevance_fn is not None:
            mask = df.apply(lambda r: bool(relevance_fn(symbol, str(r["headline"]), str(r["summary"]))), axis=1)
            df = df[mask].reset_index(drop=True)

        return df

    def batch_fetch(
        self,
        symbols: list[str],
        days: int = 7,
        max_per_source: int = 120,
        use_sources: list[str] | None = None,
        relevance_fn = None,
    ):
        """
        Multi-symbol fetch. Returns LONG format over 'symbol'.
        Each symbol is independently cached (like MarketDataTool.batch_get_prices).
        """
        import pandas as pd
        frames = []
        for s in [x.upper() for x in symbols]:
            df = self.fetch_one(
                s, days=days, max_per_source=max_per_source,
                use_sources=use_sources, relevance_fn=relevance_fn
            )
            if not df.empty:
                frames.append(df)
        out = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=self.columns())
        if not out.empty:
            out["published_utc"] = pd.to_datetime(out["published_utc"], utc=True, errors="coerce")
        return out


In [11]:
# --- NewsDataTool tests (smoke & cache behavior) ---
def test_news_all_sources():
    # 1) Fresh cache for a clean run
    test_cache = ".cache/news_all_sources_test"
    shutil.rmtree(test_cache, ignore_errors=True)

    # 2) Instantiate tool
    ndt = NewsDataTool(
        cache_dir=test_cache,
        ttl_seconds=60,            # short TTL for testing
        max_retries=2,
        pause_between_retries=0.8  # increase if rate limits hit
    )

    # 3) Symbols and sources (Yahoo + Finnhub + Polygon)
    symbols = ["AAPL","MSFT","NVDA","GOOGL","TSLA"]
    sources = ["yahoo","finnhub","polygon"]

    # 4) Fetch a decently wide window
    df = ndt.batch_fetch(
        symbols=symbols,
        days=10,                  # used by Finnhub
        max_per_source=100,       # Polygon is limit-based (up to 1000); start modest
        use_sources=sources,
        relevance_fn=None         # first fetch without filtering
    )

    print(f"Rows fetched ({'+'.join(sources)}):", len(df))
    if df.empty:
        print("No rows returned. Try increasing 'days' or 'max_per_source', or bump 'pause_between_retries' to handle rate limits.")
        return

    # 5) Ensure datetime and basic diagnostics
    df["published_utc"] = pd.to_datetime(df["published_utc"], utc=True, errors="coerce")
    assert is_datetime64_any_dtype(df["published_utc"]), "published_utc should be datetime-like"

    print("\nCounts by symbol/source:")
    display(df.groupby(["symbol","source"]).size().rename("rows").reset_index().sort_values("rows", ascending=False))

    print("\nLatest timestamp by symbol:")
    display(df.groupby("symbol")["published_utc"].max().sort_values(ascending=False))

    print("\nSample headlines (newest first):")
    display(df.sort_values("published_utc", ascending=False).head(12)[
        ["symbol","published_utc","source","publisher","headline"]
    ])

    # 6) Now apply a relevance filter (same logic your agent uses)
    ALIASES = {
        "AAPL":  ["apple","iphone","ipad","mac","tim cook","app store","vision pro"],
        "MSFT":  ["microsoft","windows","azure","xbox","satya nadella","copilot","github"],
        "NVDA":  ["nvidia","cuda","h100","blackwell","geforce","jensen huang","dgx"],
        "GOOGL": ["google","alphabet","youtube","android","sundar pichai","gemini"],
        "TSLA":  ["tesla","elon musk","model 3","model y","gigafactory","fsd"],
    }
    def relevance_fn(sym, headline, summary):
        text = f"{(headline or '').lower()} {(summary or '').lower()}"
        return any(a in text for a in ALIASES.get(sym, []))

    df_rel = df[df.apply(lambda r: relevance_fn(r["symbol"], r["headline"], r["summary"]), axis=1)].copy()
    print(f"\nRelevance-kept rows: {len(df_rel)} (from {len(df)})")
    display(df_rel.sort_values("published_utc", ascending=False).head(12)[
        ["symbol","published_utc","source","publisher","headline"]
    ])

# Run it
test_news_all_sources()


Rows fetched (yahoo+finnhub+polygon): 1005

Counts by symbol/source:


,symbol,source,rows
0,AAPL,Finnhub,100
1,AAPL,Polygon,100
3,GOOGL,Finnhub,100
6,MSFT,Finnhub,100
4,GOOGL,Polygon,100
9,NVDA,Finnhub,100
7,MSFT,Polygon,100
13,TSLA,Polygon,100
12,TSLA,Finnhub,100
10,NVDA,Polygon,100



Latest timestamp by symbol:


symbol
GOOGL   2025-10-19 23:30:00+00:00
MSFT    2025-10-19 23:30:00+00:00
NVDA    2025-10-19 23:10:00+00:00
AAPL    2025-10-19 23:01:04+00:00
TSLA    2025-10-19 22:23:00+00:00
Name: published_utc, dtype: datetime64[ns, UTC]


Sample headlines (newest first):


,symbol,published_utc,source,publisher,headline
201,MSFT,2025-10-19 23:30:00+00:00,Polygon,The Motley Fool,1 Glorious Growth Stock Down 22% You'll Regret...
603,GOOGL,2025-10-19 23:30:00+00:00,Polygon,The Motley Fool,1 Glorious Growth Stock Down 22% You'll Regret...
202,MSFT,2025-10-19 23:15:00+00:00,Polygon,The Motley Fool,1 Top Stock to Buy to Cash In on This Once-in-...
604,GOOGL,2025-10-19 23:15:00+00:00,Polygon,The Motley Fool,1 Top Stock to Buy to Cash In on This Once-in-...
402,NVDA,2025-10-19 23:10:00+00:00,Polygon,The Motley Fool,"The Smartest Growth Stock to Buy With $1,000 R..."
605,GOOGL,2025-10-19 23:10:00+00:00,Polygon,The Motley Fool,"The Smartest Growth Stock to Buy With $1,000 R..."
0,AAPL,2025-10-19 23:01:04+00:00,Polygon,The Motley Fool,Investment Advisor Goes All-In on Big Pharma S...
203,MSFT,2025-10-19 23:01:04+00:00,Polygon,The Motley Fool,Investment Advisor Goes All-In on Big Pharma S...
606,GOOGL,2025-10-19 23:01:04+00:00,Polygon,The Motley Fool,Investment Advisor Goes All-In on Big Pharma S...
804,TSLA,2025-10-19 22:23:00+00:00,Polygon,The Motley Fool,Here's What Tesla's Latest Big Move Means for ...



Relevance-kept rows: 389 (from 1005)


,symbol,published_utc,source,publisher,headline
804,TSLA,2025-10-19 22:23:00+00:00,Polygon,The Motley Fool,Here's What Tesla's Latest Big Move Means for ...
1,AAPL,2025-10-19 22:20:00+00:00,Polygon,The Motley Fool,Meet the Only Vanguard ETF That Has Turned $10...
204,MSFT,2025-10-19 22:20:00+00:00,Polygon,The Motley Fool,Meet the Only Vanguard ETF That Has Turned $10...
403,NVDA,2025-10-19 22:20:00+00:00,Polygon,The Motley Fool,Meet the Only Vanguard ETF That Has Turned $10...
408,NVDA,2025-10-19 17:15:00+00:00,Polygon,The Motley Fool,Jensen Huang Just Announced Bad News for Nvidi...
407,NVDA,2025-10-19 17:15:00+00:00,Finnhub,Yahoo,Jensen Huang Just Announced Bad News for Nvidi...
409,NVDA,2025-10-19 16:55:39+00:00,Finnhub,Yahoo,Nvidia's Huang Says on Track to Make Half-Tril...
3,AAPL,2025-10-19 16:15:00+00:00,Polygon,The Motley Fool,This Fitness Tech Stock Has Crushed Apple's 20...
414,NVDA,2025-10-19 15:41:14+00:00,Finnhub,Yahoo,Nvidia's Big Tech customers might also be its ...
805,TSLA,2025-10-19 14:30:00+00:00,Polygon,The Motley Fool,Will This Go Down as Tesla's Biggest Mistake?


### Earnings Data Tool

In [12]:
class EarningsDataTool:
    """
    Company earnings estimates + actuals with robust normalization + TTL parquet cache.
    Standardized columns:
      ['report_date','eps_estimate','eps_actual_est','revenue_estimate','revenue_actual_est',
       'fiscal_year_est','fiscal_quarter_est','eps_actual_act','revenue_actual_act',
       'fiscal_year_act','fiscal_quarter_act','source_est']
    Behavior:
      - On-disk caching (parquet) with TTL
      - Simple retries
      - Combines Finnhub estimates + SEC Edgar actuals
    """
    def __init__(
        self,
        cache_dir: str = ".cache/earnings_final",
        ttl_seconds: int = 6 * 3600,
        finnhub_key: str | None = None,
        sec_user_agent: str | None = None,
    ):
        self.cache = DiskCache(cache_dir, ttl_seconds)
        self.finnhub_key = finnhub_key or FINNHUB_KEY
        self.sec_user_agent = sec_user_agent or SEC_USER_AGENT
        self._cik_map_path = os.path.join(cache_dir, "ticker_cik.parquet")
        
        if not self.finnhub_key: print("⚠️ FINNHUB_API_KEY not set.")
        if "@" not in self.sec_user_agent: print("⚠️ SEC_USER_AGENT is not a valid email.")

    def _retry_get(self, url: str, params: dict = None) -> requests.Response | None:
        headers = {}
        if "sec.gov" in url: headers["User-Agent"] = self.sec_user_agent
        try:
            r = requests.get(url, params=params, headers=headers, timeout=20)
            r.raise_for_status()
            return r
        except requests.exceptions.RequestException as e:
            print(f"HTTP error for {url}: {e}")
            return None

    def _load_ticker_cik(self) -> pd.DataFrame:
        if os.path.exists(self._cik_map_path):
            if (time.time() - os.path.getmtime(self._cik_map_path)) < 30 * 24 * 3600:
                return pd.read_parquet(self._cik_map_path)
        url = "https://www.sec.gov/files/company_tickers.json"
        response = self._retry_get(url)
        if response is None: return pd.DataFrame()
        data = response.json()
        df = pd.DataFrame(list(data.values()))
        df = df.rename(columns={"cik_str": "cik", "ticker": "symbol"})
        df["symbol"] = df["symbol"].str.upper()
        df.to_parquet(self._cik_map_path, index=False)
        return df

    def _ticker_to_cik(self, symbol: str) -> str | None:
        df = self._load_ticker_cik()
        if df.empty: return None
        result = df[df["symbol"] == symbol.upper()]
        if not result.empty: return f"{result.iloc[0]['cik']:010d}"
        return None

    def _fetch_finnhub_estimates(self, symbol: str) -> pd.DataFrame:
        if not self.finnhub_key: return pd.DataFrame()
        today = dt.date.today()
        start_date = (today - dt.timedelta(days=730)).isoformat()
        end_date = (today + dt.timedelta(days=270)).isoformat()
        url = "https://finnhub.io/api/v1/calendar/earnings"
        params = {"from": start_date, "to": end_date, "symbol": symbol, "token": self.finnhub_key}
        response = self._retry_get(url, params)
        if response is None: return pd.DataFrame()
        data = response.json().get("earningsCalendar", [])
        if not data: return pd.DataFrame()
        df = pd.DataFrame(data)
        df = df.rename(columns={
            "date": "report_date", "epsEstimate": "eps_estimate", "epsActual": "eps_actual_est",
            "revenueEstimate": "revenue_estimate", "revenueActual": "revenue_actual_est",
            "year": "fiscal_year_est", "quarter": "fiscal_quarter_est"
        })
        df["source_est"] = "Finnhub"
        return df

    def _fetch_edgar_actuals(self, symbol: str) -> pd.DataFrame:
        cik = self._ticker_to_cik(symbol)
        if not cik: return pd.DataFrame()
        url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
        response = self._retry_get(url)
        if response is None: return pd.DataFrame()
        facts = response.json().get("facts", {}).get("us-gaap", {})
        revenue_tag = facts.get("Revenues") or facts.get("SalesRevenueNet") or {}
        eps_tag = facts.get("EarningsPerShareDiluted", {})
        def extract_series(tag_data):
            rows = []
            for unit in tag_data.get("units", {}).values():
                for fact in unit:
                    if fact.get("form") in ["10-Q", "10-K"]:
                        rows.append({"report_date": pd.to_datetime(fact["end"]), "value": fact["val"], "fy": fact["fy"], "fp": fact["fp"]})
            df = pd.DataFrame(rows)
            if not df.empty:
                df = df.sort_values("report_date").drop_duplicates(subset=["fy", "fp"], keep="last")
            return df
        df_rev = extract_series(revenue_tag)
        df_eps = extract_series(eps_tag)
        if df_rev.empty or df_eps.empty: return pd.DataFrame()
        df = pd.merge(df_rev, df_eps, on=["fy", "fp"], suffixes=('_rev', '_eps'))
        df = df.rename(columns={
            "report_date_rev": "report_date", "value_rev": "revenue_actual_act",
            "value_eps": "eps_actual_act", "fy": "fiscal_year_act", "fp": "fiscal_quarter_act"
        })
        df = df[df["fiscal_quarter_act"].str.startswith("Q")].copy()
        df["fiscal_quarter_act"] = df["fiscal_quarter_act"].str.replace("Q", "").astype(int)
        df["source_act"] = "EDGAR"
        return df

    def fetch_one(self, symbol: str) -> pd.DataFrame:
        cache_key = f"earnings_final_v1::{symbol}"
        cached_df = self.cache.get(cache_key)
        if cached_df is not None: return cached_df

        df_est_raw = self._fetch_finnhub_estimates(symbol)
        df_act_raw = self._fetch_edgar_actuals(symbol)

        if df_est_raw.empty or df_act_raw.empty:
            return df_est_raw if not df_est_raw.empty else df_act_raw

        # --- FIX 1: Select only the columns you need before merging ---
        est_cols = ["report_date", "eps_estimate", "revenue_estimate", "fiscal_year_est", "fiscal_quarter_est", "source_est"]
        act_cols = ["report_date", "eps_actual_act", "revenue_actual_act", "fiscal_year_act", "fiscal_quarter_act", "source_act"]
        df_est = df_est_raw[est_cols].copy()
        df_act = df_act_raw[act_cols].copy()

        df_est['report_date'] = pd.to_datetime(df_est['report_date'], errors='coerce', utc=True)
        df_act['report_date'] = pd.to_datetime(df_act['report_date'], errors='coerce', utc=True)
        df_est = df_est.sort_values('report_date')
        df_act = df_act.sort_values('report_date')

        df_merged = pd.merge_asof(
            df_est, df_act, on='report_date', direction='backward',
            tolerance=pd.Timedelta(days=120)
        )

        df_merged['eps_actual'] = df_merged['eps_actual_act']
        df_merged['revenue_actual'] = df_merged['revenue_actual_act']
        df_merged['fiscal_year'] = df_merged['fiscal_year_act'].fillna(df_merged['fiscal_year_est'])
        df_merged['fiscal_quarter'] = df_merged['fiscal_quarter_act'].fillna(df_merged['fiscal_quarter_est'])

        for col in ["eps_estimate", "eps_actual", "revenue_estimate", "revenue_actual"]:
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

        df_merged["eps_surprise"] = df_merged["eps_actual"] - df_merged["eps_estimate"]
        df_merged["rev_surprise"] = df_merged["revenue_actual"] - df_merged["revenue_estimate"]
        df_merged["beat_flag"] = df_merged["eps_surprise"] > 0
        
        df_merged['fiscal_year'] = df_merged['fiscal_year'].astype('Int64')
        df_merged['fiscal_quarter'] = df_merged['fiscal_quarter'].astype('Int64')

        final_cols = [
            "symbol", "report_date", "eps_estimate", "eps_actual", "eps_surprise",
            "revenue_estimate", "revenue_actual", "rev_surprise", "beat_flag",
            "fiscal_year", "fiscal_quarter", "source_est", "source_act"
        ]
        df_merged["symbol"] = symbol.upper()
        df_final = df_merged.reindex(columns=final_cols).sort_values("report_date", ascending=False, na_position='last').reset_index(drop=True)
        
        self.cache.set(cache_key, df_final)
        return df_final

    def batch_fetch(self, symbols: list[str]) -> pd.DataFrame:
        all_dfs = [self.fetch_one(s) for s in symbols]
        valid_dfs = [df for df in all_dfs if df is not None and not df.empty]
        if not valid_dfs: return pd.DataFrame()
        return pd.concat(valid_dfs, ignore_index=True)

In [13]:
# --- How to use the refactored tool ---
print("--- Testing the Refactored EarningsDataTool (Finnhub + SEC) ---")

# Make sure to set your API keys as environment variables
# For example: FINNHUB_API_KEY="your_key"
# For example: SEC_USER_AGENT="Your Name you@example.com"
tool = EarningsDataTool()

earnings_df = tool.batch_fetch(["NVDA", "AAPL", "TSLA"])

if not earnings_df.empty:
    print(f"\n✅ Successfully fetched and merged data for {earnings_df['symbol'].nunique()} symbols.")
    print("--- Sample of Merged Data ---")
    display(earnings_df.head(10))
else:
    print("\n❌ Could not fetch any earnings data. Check API keys and network connection.")

--- Testing the Refactored EarningsDataTool (Finnhub + SEC) ---

✅ Successfully fetched and merged data for 3 symbols.
--- Sample of Merged Data ---


,symbol,report_date,eps_estimate,eps_actual,eps_surprise,revenue_estimate,revenue_actual,rev_surprise,beat_flag,fiscal_year,fiscal_quarter,source_est,source_act,eps_actual_est,hour,fiscal_quarter_est,revenue_actual_est,fiscal_year_est
0,NVDA,2026-05-26 00:00:00+00:00,1.5242,NaN,NaN,65411105088,NaN,NaN,False,2027,1,Finnhub,NaN,NaN,NaN,NaN,NaN,NaN
1,NVDA,2026-02-24 00:00:00+00:00,1.4456,NaN,NaN,62366819952,NaN,NaN,False,2026,4,Finnhub,NaN,NaN,NaN,NaN,NaN,NaN
2,NVDA,2025-11-19 00:00:00+00:00,1.2651,1.08,-0.1851,55753113351,4.674300e+10,-9.010113e+09,False,2026,2,Finnhub,EDGAR,NaN,NaN,NaN,NaN,NaN
3,AAPL,2026-04-29,1.8424,NaN,NaN,103726965355,NaN,NaN,NaN,<NA>,<NA>,Finnhub,NaN,None,amc,2.0,None,2026.0
4,AAPL,2026-01-28,2.5411,NaN,NaN,133684531371,NaN,NaN,NaN,<NA>,<NA>,Finnhub,NaN,None,amc,1.0,None,2026.0
5,AAPL,2025-10-30,1.7924,NaN,NaN,103706233519,NaN,NaN,NaN,<NA>,<NA>,Finnhub,NaN,None,amc,4.0,None,2025.0
6,TSLA,2026-04-20 00:00:00+00:00,0.4534,NaN,NaN,23522120692,NaN,NaN,False,2026,1,Finnhub,NaN,NaN,NaN,NaN,NaN,NaN
7,TSLA,2026-01-27 00:00:00+00:00,0.4810,NaN,NaN,25879316580,NaN,NaN,False,2025,4,Finnhub,NaN,NaN,NaN,NaN,NaN,NaN
8,TSLA,2025-10-22 00:00:00+00:00,0.5399,0.33,-0.2099,26589014709,2.249600e+10,-4.093015e+09,False,2025,2,Finnhub,EDGAR,NaN,NaN,NaN,NaN,NaN


## Visualization

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

class VisualizationTool:
    """
    An upgraded tool to create and save visualizations, including comparative
    and economic context charts.
    """
    def __init__(self, save_dir: str = "reports/images"):
        self.save_dir = save_dir
        os.makedirs(self.save_dir, exist_ok=True)
        sns.set_theme(style="whitegrid")

    def plot_price_history(self, df: pd.DataFrame, symbol: str) -> str | None:
        if df is None or df.empty or 'date' not in df.columns or 'close' not in df.columns:
            print(f"   - Skipping price chart for {symbol}: insufficient data.")
            return None
        
        plt.figure(figsize=(12, 6))
        plt.plot(df['date'], df['close'], label=f'{symbol} Close Price', color='blue')
        if 'sma_20' in df.columns: plt.plot(df['date'], df['sma_20'], label='20-Day SMA', color='orange', linestyle='--')
        if 'sma_50' in df.columns: plt.plot(df['date'], df['sma_50'], label='50-Day SMA', color='red', linestyle='--')
        plt.title(f'{symbol} Price History', fontsize=16)
        plt.xlabel('Date'); plt.ylabel('Price (USD)'); plt.legend(); plt.grid(True)
        
        filepath = os.path.join(self.save_dir, f"{symbol}_price_history.png")
        plt.savefig(filepath); plt.close()
        print(f"   - Chart saved to: {filepath}")
        return filepath

    def plot_earnings_surprise(self, df: pd.DataFrame, symbol: str) -> str | None:
        if df is None or df.empty or 'eps_surprise' not in df.columns:
            print(f"   - Skipping earnings chart for {symbol}: no surprise data.")
            return None
        
        plot_df = df.dropna(subset=['eps_surprise']).sort_values('report_date').tail(16)
        if plot_df.empty:
            print(f"   - Skipping earnings chart for {symbol}: no valid surprise data points.")
            return None
            
        plt.figure(figsize=(12, 6))
        colors = ['green' if x >= 0 else 'red' for x in plot_df['eps_surprise']]
        plot_df['report_date_str'] = plot_df['report_date'].dt.strftime('%Y-%m-%d')
        plt.bar(plot_df['report_date_str'], plot_df['eps_surprise'], color=colors)
        plt.title(f'{symbol} Quarterly EPS Surprise', fontsize=16)
        plt.xlabel('Report Date'); plt.ylabel('EPS Surprise (USD)'); plt.xticks(rotation=45)
        plt.axhline(0, color='black', linewidth=0.8, linestyle='--'); plt.tight_layout()
        
        filepath = os.path.join(self.save_dir, f"{symbol}_eps_surprise.png")
        plt.savefig(filepath); plt.close()
        print(f"   - Chart saved to: {filepath}")
        return filepath

    def plot_comparative_price_history(self, data_dict: dict[str, pd.DataFrame]) -> str | None:
        """Plots the normalized price history for multiple stocks."""
        plt.figure(figsize=(12, 6))
        for symbol, df in data_dict.items():
            if df is not None and not df.empty and 'close' in df.columns:
                # Normalize prices to show percentage change from the start
                normalized_price = (df['close'] / df['close'].iloc[0]) * 100
                plt.plot(df['date'], normalized_price, label=f'{symbol}')

        plt.title('Comparative Stock Performance (Normalized)', fontsize=16)
        plt.xlabel('Date'); plt.ylabel('Normalized Price (Start = 100)'); plt.legend(); plt.grid(True)
        
        filepath = os.path.join(self.save_dir, "comparative_price_history.png")
        plt.savefig(filepath); plt.close()
        print(f"   - Chart saved to: {filepath}")
        return filepath

    def plot_stock_vs_economic_series(self, stock_df: pd.DataFrame, econ_df: pd.DataFrame, symbol: str, econ_series_id: str) -> str | None:
        """Plots a stock's price against an economic series using a dual axis."""
        if stock_df is None or econ_df is None or stock_df.empty or econ_df.empty:
            return None

        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # Plot stock price on the left axis
        color = 'tab:blue'
        ax1.set_xlabel('Date')
        ax1.set_ylabel(f'{symbol} Price (USD)', color=color)
        ax1.plot(stock_df['date'], stock_df['close'], color=color)
        ax1.tick_params(axis='y', labelcolor=color)

        # Create a second y-axis for the economic data
        ax2 = ax1.twinx()
        color = 'tab:red'
        ax2.set_ylabel(econ_series_id, color=color)
        ax2.plot(econ_df['date'], econ_df[econ_series_id], color=color, linestyle='--')
        ax2.tick_params(axis='y', labelcolor=color)

        plt.title(f'{symbol} Price vs. {econ_series_id}', fontsize=16)
        fig.tight_layout()
        
        filepath = os.path.join(self.save_dir, f"{symbol}_vs_{econ_series_id}.png")
        plt.savefig(filepath); plt.close()
        print(f"   - Chart saved to: {filepath}")
        return filepath


# Agent

In [15]:
import openai
import json
import base64
import pandas as pd
from IPython.display import display, HTML, Markdown, Image

# The 'markdown' library is used for better report formatting.
# If you don't have it, run: !pip install markdown
try:
    import markdown
except ImportError:
    markdown = None

class InvestmentResearchAgent:
    """
    An autonomous agent that plans, executes, and refines investment research,
    producing a final HTML report with text, tables, and visualizations.
    """
    def __init__(self, model_name: str = "gpt-4o-mini"):
        self.client = openai.OpenAI()
        self.model = model_name
        
        print("Initializing tools...")
        self.market_tool = MarketDataTool()
        self.earnings_tool = EarningsDataTool()
        self.economic_tool = EconomicDataTool()
        self.viz_tool = VisualizationTool()
        self.news_tool = NewsDataTool()
        self.memory_tool = MemoryStore()
        print("Tools initialized. Agent is ready. 🚀")

    def _invoke_llm(self, messages: list, temperature: float = 0.1, json_mode: bool = False):
        try:
            response = self.client.chat.completions.create(
                model=self.model, messages=messages, temperature=temperature,
                response_format={"type": "json_object"} if json_mode else None
            )
            return response.choices[0].message.content
        except Exception as e: 
            print(f"Error invoking LLM: {e}")
            return None

    def _plan(self, topic: str) -> list[dict]:
        """Guideline: "Plans its research steps" """
        system_prompt = "You are a meticulous financial research planner. Your only function is to output a valid JSON array of objects."
        user_prompt = f"""
        Create a step-by-step research plan for the topic: "{topic}".
        Use a logical sequence. Fetch data before generating charts or tables.

        Available tasks:
        - get_news (symbol): For news on a stock ticker (e.g., "AAPL").
        - process_news (symbol): To summarize news after getting it.
        - get_market_data (symbol): For price history of a stock ticker.
        - get_earnings (symbol): For earnings history of a stock ticker.
        - get_economic_data (series_ids): For economic data series (e.g., ["CPIAUCSL", "GDP"]).
        - generate_price_chart (symbol): Creates a price chart for one stock.
        - generate_earnings_chart (symbol): Creates an EPS surprise chart for one stock.
        - generate_comparative_table (symbols): Creates a summary table for a LIST of stocks.
        - generate_stock_vs_economic_chart (symbol, series_id): Compares a stock to an economic series.
        - generate_comparative_price_chart (symbols): Creates a comparative chart for a LIST of stocks.

        IMPORTANT RULES:
        1. Identify entities correctly. 'AAPL' is a stock symbol. 'CPIAUCSL' is an economic series_id.
        2. For economic data like 'CPIAUCSL', you MUST use the 'get_economic_data' task. NEVER use 'get_market_data' for economic series.
        3. If the topic asks to "correlate" a stock with an economic indicator, you MUST include 'get_economic_data' and 'generate_stock_vs_economic_chart' in your plan.

        Your output must be a valid JSON array.
        """
        messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
        plan_str = self._invoke_llm(messages, json_mode=True)
        if not plan_str: 
            print("Error: LLM returned an empty response for the plan.")
            return []
        try:
            plan_data = json.loads(plan_str)
            if isinstance(plan_data, list): return plan_data
            if isinstance(plan_data, dict):
                for value in plan_data.values():
                    if isinstance(value, list): return value
            print(f"Error: Could not find a list in the JSON plan: {plan_data}")
            return []
        except (json.JSONDecodeError, TypeError) as e:
            print(f"Error: Failed to parse JSON plan. Error: {e}. Raw response: {plan_str}")
            return []

    def _process_news_chain(self, symbol: str, news_df: pd.DataFrame) -> dict:
        """Workflow Pattern: "Prompt Chaining" """
        if news_df is None or news_df.empty:
            return {"summary": "No news to process."}
        
        articles_text = "\n\n---\n\n".join(
            [f"Headline: {row['headline']}\nSummary: {row['summary']}" for _, row in news_df.head(10).iterrows()]
        )
        
        chain_prompt = f"""
        You are a news analyst. For the following articles about {symbol}, perform these tasks:
        1. Classify sentiment (Positive, Negative, Neutral).
        2. Extract 3-5 key points.
        3. Summarize the key news themes.

        Return a valid JSON object with "sentiment", "key_points" (a list), and "summary" (a string).

        Articles:\n---\n{articles_text}\n---
        """
        result_str = self._invoke_llm([{"role": "user", "content": chain_prompt}], json_mode=True)
        try:
            return json.loads(result_str)
        except (json.JSONDecodeError, TypeError):
            return {"summary": "Failed to process news."}

    def _reflect_and_refine(self, initial_analysis: str, topic: str) -> str:
        """Workflow Pattern: "Evaluator-Optimizer" """
        print("\nStep 3a: 🧐 Critiquing initial analysis...")
        critique_prompt = f"""Critique the following research report draft for the topic '{topic}'. 
        Check for clarity, objectivity, and completeness. 
        Provide specific, actionable suggestions for improvement as a Markdown numbered list.
        \n\nDraft:\n{initial_analysis}"""
        critique = self._invoke_llm([{"role": "user", "content": critique_prompt}])
        print(f"--- CRITIQUE ---\n{critique or 'No critique generated.'}")

        print("\nStep 3b: ✍️ Refining analysis based on critique...")
        refine_prompt = f"""Rewrite and improve the report draft based on the critique. 
        Produce the final, polished version of the analysis. 
        Format your entire response in Markdown, using headings, lists, and bold text for clarity.
        \n\nOriginal Draft:\n{initial_analysis}\n\nCritique:\n{critique}"""
        refined_analysis = self._invoke_llm([{"role": "user", "content": refine_prompt}])
        return refined_analysis or initial_analysis

    def _create_html_report(self, topic: str, analysis_text: str, plan: list, results: dict) -> str:
        """Assembles the final HTML report with text, charts, and tables."""
        def image_to_base64(path):
            if not path or not os.path.exists(path): return ""
            with open(path, "rb") as img_file: return base64.b64encode(img_file.read()).decode('utf-8')

        if markdown:
            analysis_html = markdown.markdown(analysis_text) if analysis_text else ""
        else:
            analysis_html = f"<div>{analysis_text.replace('\n', '<br>')}</div>" if analysis_text else ""

        html_parts = [analysis_html]
        
        for step in plan:
            task = step.get("task")
            
            if "chart" in task:
                chart_path = None
                if task == "generate_price_chart": chart_path = results.get(f"{step.get('symbol')}_price_chart")
                elif task == "generate_earnings_chart": chart_path = results.get(f"{step.get('symbol')}_earnings_chart")
                elif task == "generate_stock_vs_economic_chart":
                    chart_path = results.get(f"{step.get('symbol')}_vs_{step.get('series_id')}_chart")
                elif task == "generate_comparative_price_chart":
                    chart_path = results.get("comparative_price_chart")

                if chart_path:
                    html_parts.append(f'<img src="data:image/png;base64,{image_to_base64(chart_path)}" style="width:100%;height:auto;max-width:800px;margin-top:20px;">')
            
            elif task == "generate_comparative_table":
                table_df = results.get("comparative_table")
                if table_df is not None and not table_df.empty:
                    html_parts.append(f"<h3 style='margin-top:20px;'>Comparative Metrics</h3>")
                    html_parts.append(table_df.to_html(index=False, classes='table', border=0))

        body_content = "".join(html_parts)
        return f"""<html><head><title>{topic}</title><style>
            body{{font-family:Arial,sans-serif;line-height:1.6;}}
            h1,h2,h3{{color:#333;}}
            img{{border:1px solid #ddd;border-radius:5px;}}
            table{{width:100%;border-collapse:collapse;margin-top:20px;}}
            th,td{{padding:8px;text-align:left;border-bottom:1px solid #ddd;}}
            th{{background-color:#f2f2f2;}}
            </style></head><body><h1>Research Report: {topic}</h1>{body_content}</body></html>"""

    def run(self, topic: str):
        """Main orchestrator for the agent."""
        
        print("Step 1: 🧠 Creating a research plan...")
        plan = self._plan(topic)
        if not plan:
            print("Could not create a valid plan. Aborting.")
            return

        print("Plan created:")
        for i, step in enumerate(plan):
            task_info = step.get('task', 'N/A')
            details = step.get('symbol') or step.get('symbols') or step.get('series_ids', [])
            print(f"  {i+1}. {task_info} for {details}")

        symbols_in_plan = list(set([s for step in plan if (s := step.get("symbol"))]))

        print("\nStep 2: 🛠️ Executing the plan (Routing)...")
        results_store = {}
        for step in plan:
            task = step.get("task")
            symbol = step.get("symbol")
            print(f"  Executing task: {task}...")
            
            if task == "get_news":
                results_store[f"{symbol}_news_raw"] = self.news_tool.fetch_one(symbol, days=14)
            elif task == "process_news":
                raw_news = results_store.get(f"{symbol}_news_raw")
                results_store[f"{symbol}_news_processed"] = self._process_news_chain(symbol, raw_news)
            elif task == "get_market_data":
                results_store[f"{symbol}_market_data"] = self.market_tool.get_price_panel(ticker=symbol, period="2y")
            elif task == "get_earnings":
                results_store[f"{symbol}_earnings_data"] = self.earnings_tool.fetch_one(symbol=symbol)
            elif task == "get_economic_data":
                series_ids = step.get("series_ids", [])
                results_store["economic_data"] = self.economic_tool.get_series(series_ids=series_ids)
            elif task == "generate_price_chart":
                df = results_store.get(f"{symbol}_market_data")
                chart_path = self.viz_tool.plot_price_history(df, symbol)
                results_store[f"{symbol}_price_chart"] = chart_path
                #if chart_path:
                #    display(Image(filename=chart_path))
            elif task == "generate_earnings_chart":
                df = results_store.get(f"{symbol}_earnings_data")
                chart_path = self.viz_tool.plot_earnings_surprise(df, symbol)
                results_store[f"{symbol}_earnings_chart"] = chart_path
                #if chart_path:
                #    display(Image(filename=chart_path))
            elif task == "generate_stock_vs_economic_chart":
                series_id = step.get("series_id")
                stock_df = results_store.get(f"{symbol}_market_data")
                econ_df = results_store.get("economic_data")
                chart_path = self.viz_tool.plot_stock_vs_economic_series(stock_df, econ_df, symbol, series_id)
                results_store[f"{symbol}_vs_{series_id}_chart"] = chart_path
                #if chart_path:
                #    display(Image(filename=chart_path))
            elif task == "generate_comparative_price_chart":
                symbols = step.get("symbols", [])
                data_dict = {s: results_store.get(f"{s}_market_data") for s in symbols}
                chart_path = self.viz_tool.plot_comparative_price_history(data_dict)
                results_store["comparative_price_chart"] = chart_path
                #if chart_path:
                #    display(Image(filename=chart_path))
            elif task == "generate_comparative_table":
                symbols = step.get("symbols", [])
                table_data = []
                for s in symbols:
                    market_df, earnings_df = results_store.get(f"{s}_market_data"), results_store.get(f"{s}_earnings_data")
                    row = {"Symbol": s}
                    if market_df is not None and not market_df.empty: row["Latest Close"] = f"${market_df['close'].iloc[-1]:.2f}"
                    if earnings_df is not None and not earnings_df.empty and 'eps_surprise' in earnings_df.columns:
                        latest_surprise = earnings_df.dropna(subset=['eps_surprise']).iloc[0] if not earnings_df.dropna(subset=['eps_surprise']).empty else None
                        if latest_surprise is not None: row["Latest EPS Surprise"] = f"{latest_surprise['eps_surprise']:.4f}"
                    table_data.append(row)
                results_store["comparative_table"] = pd.DataFrame(table_data)

        print("\nStep 3: ✍️ Generating and Refining Analysis...")
        synthesis_prompt = f"""You are a senior investment analyst. Write a comprehensive analysis for the topic: '{topic}'. 
        Synthesize all the information provided below into a coherent report. 
        Format your entire response in Markdown, including an executive summary and detailed sections.
        \n\nAvailable Data:\n---"""
        for key, value in results_store.items():
            synthesis_prompt += f"\n### {key}\n"
            if isinstance(value, pd.DataFrame): synthesis_prompt += value.head().to_markdown() + "\n"
            else: synthesis_prompt += str(value) + "\n"
        initial_analysis = self._invoke_llm([{"role": "user", "content": synthesis_prompt}]) or "Analysis could not be generated."
        final_analysis = self._reflect_and_refine(initial_analysis, topic)

        print("\nStep 4: 🎨 Assembling final HTML report (for saving)...")
        final_html = self._create_html_report(topic, final_analysis, plan, results_store)
        
        filename = topic.lower().replace(" ", "_").replace("/", "")[:50] + ".html"
        with open(filename, "w", encoding="utf-8") as f: f.write(final_html)
        print(f"\n--- 💾 Report saved to {filename} ---")
        
        # --- FIX: Display the final analysis as formatted Markdown in the notebook ---
        print("\n--- ✅ FINAL REPORT ---")
        display(Markdown(final_analysis))

        print("\nStep 5: 💾 Learning from the analysis...")
        memory_prompt = f"Based on the analysis for '{topic}', write a single, concise sentence summarizing the most important takeaway for future runs."
        memory_note = self._invoke_llm([{"role": "user", "content": memory_prompt}])
        if memory_note:
            for symbol in symbols_in_plan:
                self.memory_tool.add_note(symbol, memory_note)



## Run the Agent

In [16]:
# A topic that will trigger the stock vs. economic chart
ECONOMIC_TOPIC = "Analyze how Apple's (AAPL) stock price correlates with US inflation (CPIAUCSL)."

agent = InvestmentResearchAgent()
agent.run(ECONOMIC_TOPIC)

Initializing tools...
Tools initialized. Agent is ready. 🚀
Step 1: 🧠 Creating a research plan...
Plan created:
  1. get_news for AAPL
  2. process_news for AAPL
  3. get_market_data for AAPL
  4. get_economic_data for ['CPIAUCSL']
  5. generate_stock_vs_economic_chart for AAPL

Step 2: 🛠️ Executing the plan (Routing)...
  Executing task: get_news...
  Executing task: process_news...
  Executing task: get_market_data...
  Executing task: get_economic_data...
  Executing task: generate_stock_vs_economic_chart...
   - Chart saved to: reports/images\AAPL_vs_CPIAUCSL.png

Step 3: ✍️ Generating and Refining Analysis...

Step 3a: 🧐 Critiquing initial analysis...
--- CRITIQUE ---
Here’s a critique of your research report draft on the correlation between Apple's stock price and US inflation, focusing on clarity, objectivity, and completeness. Below are specific, actionable suggestions for improvement:

### Critique and Suggestions for Improvement

1. **Clarify the Time Frame of Data**:
   - **S

# Analysis of Apple's (AAPL) Stock Price Correlation with US Inflation (CPIAUCSL)

## Executive Summary

This report examines the correlation between Apple's stock price (AAPL) and US inflation, as measured by the Consumer Price Index for All Urban Consumers (CPIAUCSL). Utilizing recent market data, economic indicators, and news sentiment, the analysis reveals a complex relationship where inflationary pressures can impact consumer spending, thereby affecting Apple's revenue and stock performance. Notably, Apple's innovative product launches and strategic partnerships may mitigate some adverse effects of inflation. This report aims to provide investors with insights into how inflation influences AAPL's stock price and offers recommendations for navigating these economic conditions.

## 1. Introduction

Apple Inc. (AAPL) stands as one of the largest technology companies globally, renowned for its innovative products and services. Understanding the impact of external economic factors, such as inflation, on its stock price is crucial for investors. This report investigates the correlation between AAPL's stock price and US inflation, focusing on the Consumer Price Index (CPIAUCSL) as a key economic indicator.

## 2. Data Overview

### 2.1 AAPL Market Data

The following table presents recent market data for AAPL, covering the period from October 18 to October 24, 2023:

| Date       | Open    | High    | Low     | Close   | Volume    |
|------------|---------|---------|---------|---------|-----------|
| 2023-10-18 | 173.877 | 175.857 | 173.411 | 174.134 | 54,764,400|
| 2023-10-19 | 174.332 | 176.115 | 173.491 | 173.758 | 59,302,900|
| 2023-10-20 | 173.610 | 173.718 | 170.965 | 171.203 | 64,244,000|
| 2023-10-23 | 169.252 | 172.322 | 168.282 | 171.322 | 55,980,100|
| 2023-10-24 | 171.371 | 171.985 | 169.787 | 171.758 | 43,816,600|

### 2.2 US Inflation Data (CPIAUCSL)

The following table presents recent CPI data, which indicates a steady increase in inflation:

| Date       | CPIAUCSL |
|------------|----------|
| 2023-09-01 | 323.364  |
| 2023-08-01 | 322.132  |
| 2023-07-01 | 321.500  |
| 2023-06-01 | 320.580  |
| 2023-05-01 | 320.321  |

*Note: The CPI data has been updated to reflect the correct timeframe for correlation analysis with AAPL stock prices.*

## 3. Correlation Analysis

### 3.1 Historical Correlation

The correlation between AAPL's stock price and CPIAUCSL can be assessed through historical trends. Generally, rising inflation can lead to increased costs for consumers, potentially reducing discretionary spending on premium products like those offered by Apple. However, Apple's strong brand loyalty and innovative product offerings may buffer it against inflationary pressures.

### 3.2 Recent Trends

Recent developments highlight positive market sentiment surrounding Apple, driven by its inclusion in the top-performing Vanguard Information Technology ETF and the launch of its new M5 chip. These factors suggest that despite inflationary pressures, Apple's innovative strategies may continue to attract investors and consumers.

### 3.3 Sentiment Analysis

Current sentiment surrounding AAPL is predominantly positive, with indicators pointing to strong performance in the tech sector and growth potential, particularly in artificial intelligence (AI). This favorable sentiment may counteract some negative impacts of inflation on stock performance.

## 4. Implications of Inflation on AAPL

### 4.1 Consumer Spending

Inflation can lead to higher prices for goods and services, which may reduce consumer spending power. For Apple, this could translate to lower sales volumes if consumers opt for cheaper alternatives. However, Apple's premium positioning may allow it to maintain sales despite inflation.

### 4.2 Cost Structure

Rising costs for materials and labor can impact Apple's profit margins. The company must manage its supply chain effectively to mitigate these risks. Innovations, such as the M5 chip, may help maintain competitive advantages and justify premium pricing.

### 4.3 Strategic Partnerships

Apple's acquisition of exclusive US Formula 1 broadcast rights starting in 2026 represents a strategic move to diversify revenue streams. Such partnerships can enhance brand visibility and attract new customers, potentially offsetting inflation's impact.

## 5. Conclusion

The correlation between AAPL's stock price and US inflation (CPIAUCSL) is multifaceted. While inflation poses challenges, Apple's strong brand, innovative products, and strategic initiatives may help sustain its stock performance. Investors should closely monitor inflation trends and Apple's responses to these economic pressures to make informed investment decisions.

## 6. Recommendations

1. **Monitor Inflation Trends**: Keep an eye on CPI data and other economic indicators such as the Producer Price Index (PPI) and consumer sentiment indices that may affect consumer spending.
2. **Evaluate Apple's Innovations**: Assess the impact of new product launches and strategic partnerships on Apple's market position and revenue streams.
3. **Diversify Investments**: Consider diversifying portfolios to mitigate risks associated with inflation and market volatility, focusing on sectors that may benefit from inflationary environments.

![AAPL vs CPIAUCSL](reports/images/AAPL_vs_CPIAUCSL.png)

## Glossary

- **Correlation**: A statistical measure that describes the extent to which two variables change together.
- **CPIAUCSL**: Consumer Price Index for All Urban Consumers, a measure of inflation.
- **Discretionary Spending**: Non-essential expenditures that consumers can adjust based on their financial situation.

By addressing the critiques and enhancing the clarity, objectivity, and completeness of this report, we provide a more informative and engaging analysis for investors interested in AAPL's stock performance in relation to inflation.


Step 5: 💾 Learning from the analysis...
   - Memory added for AAPL


C:\Users\SyedM\AppData\Local\Temp\ipykernel_34924\417482173.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


In [17]:
# Define a research topic that requires visualizations
VISUAL_RESEARCH_TOPIC = "Create a visual report on the stock price and earnings history for NVIDIA (NVDA), Apple (APPL) and Google (GOOGL). Include charts and analysis."

# Instantiate the new version of the agent
agent = InvestmentResearchAgent()

# Run the full workflow
agent.run(VISUAL_RESEARCH_TOPIC)

Initializing tools...
Tools initialized. Agent is ready. 🚀
Step 1: 🧠 Creating a research plan...
Plan created:
  1. get_market_data for NVDA
  2. get_earnings for NVDA
  3. get_market_data for AAPL
  4. get_earnings for AAPL
  5. get_market_data for GOOGL
  6. get_earnings for GOOGL
  7. generate_price_chart for NVDA
  8. generate_earnings_chart for NVDA
  9. generate_price_chart for AAPL
  10. generate_earnings_chart for AAPL
  11. generate_price_chart for GOOGL
  12. generate_earnings_chart for GOOGL
  13. generate_comparative_table for ['NVDA', 'AAPL', 'GOOGL']

Step 2: 🛠️ Executing the plan (Routing)...
  Executing task: get_market_data...
  Executing task: get_earnings...
  Executing task: get_market_data...
  Executing task: get_earnings...
  Executing task: get_market_data...
  Executing task: get_earnings...
  Executing task: generate_price_chart...
   - Chart saved to: reports/images\NVDA_price_history.png
  Executing task: generate_earnings_chart...
   - Chart saved to: repor

# Comprehensive Analysis of NVIDIA (NVDA), Apple (AAPL), and Google (GOOGL)

## Executive Summary
This report presents a detailed visual and analytical overview of the stock price and earnings history for NVIDIA (NVDA), Apple (AAPL), and Google (GOOGL). It includes recent market data, earnings surprises, and visual representations of stock price trends. A comparative table highlights the latest closing prices and earnings surprises for each company, providing insights into their financial performance and market positioning.

---

## 1. Stock Price Analysis

### 1.1 NVIDIA (NVDA)
![NVIDIA Price History](reports/images/NVDA_price_history.png)

- **Latest Close**: $183.22
- **Recent Price Movement**: NVDA has exhibited volatility, with a notable high of $43.6713 and a low of $41.0538 in recent trading sessions. This fluctuation may be attributed to market reactions to earnings reports and industry developments.
- **Volume Trends**: The trading volume peaked at **627,294,000 shares** on October 18, 2023, indicating significant investor interest.

### 1.2 Apple (AAPL)
![Apple Price History](reports/images/AAPL_price_history.png)

- **Latest Close**: $252.29
- **Recent Price Movement**: AAPL has maintained a stable price range, with a high of $176.115 and a low of $170.965 in recent days. This stability may reflect strong consumer demand and positive market sentiment.
- **Volume Trends**: Trading volume peaked at **64,244,000 shares** on October 20, 2023, suggesting consistent investor engagement.

### 1.3 Google (GOOGL)
![Google Price History](reports/images/GOOGL_price_history.png)

- **Latest Close**: $253.30
- **Recent Price Movement**: GOOGL has experienced fluctuations, with a high of $139.756 and a low of $134.155 recently. These movements may be influenced by competitive pressures and regulatory news.
- **Volume Trends**: The trading volume peaked at **44,814,300 shares** on October 24, 2023, indicating moderate investor activity.

---

## 2. Earnings History

### 2.1 NVIDIA (NVDA)
![NVIDIA EPS Surprise](reports/images/NVDA_eps_surprise.png)

- **Latest EPS Estimate**: $1.5242 (upcoming report on May 26, 2026)
- **Latest EPS Actual**: $1.08 (reported on November 19, 2025)
- **EPS Surprise**: -0.1851, indicating that actual earnings fell short of estimates, which may impact investor confidence.

### 2.2 Apple (AAPL)
- **Latest EPS Estimate**: $1.8424 (upcoming report on April 29, 2026)
- **Latest Revenue Estimate**: $103.73 billion for Q2 2026.
- **EPS Actual**: Data is expected to be released soon, with implications for future stock performance.

### 2.3 Google (GOOGL)
- **Latest EPS Estimate**: $2.5226 (upcoming report on April 22, 2026)
- **Latest Revenue Estimate**: $103.79 billion for Q1 2026.
- **EPS Actual**: Data is forthcoming, and its release will be critical for assessing market expectations.

---

## 3. Comparative Analysis

| Symbol | Latest Close | Latest EPS Surprise |
|--------|--------------|---------------------|
| NVDA   | $183.22      | -0.1851             |
| AAPL   | $252.29      | N/A                 |
| GOOGL  | $253.30      | N/A                 |

### Insights:
- **Market Positioning**: AAPL and GOOGL are currently trading at higher prices compared to NVDA, suggesting that investors may perceive them as more stable investments based on current pricing.
- **Earnings Performance**: NVDA's recent earnings report showed a negative surprise, which could affect investor sentiment moving forward. In contrast, the upcoming earnings reports for AAPL and GOOGL will be crucial in shaping market expectations.

---

## 4. Broader Market Context
The performance of NVDA, AAPL, and GOOGL is influenced by various external factors, including:

- **Economic Indicators**: Inflation rates, interest rates, and consumer spending trends can significantly impact tech stocks.
- **Industry Trends**: The tech sector is experiencing rapid advancements, particularly in AI and cloud computing, which may affect the competitive landscape.
- **Regulatory Environment**: Ongoing scrutiny from regulators can impact stock performance, particularly for companies like Google.

---

## Conclusion
The analysis of NVIDIA, Apple, and Google reveals distinct trends in stock prices and earnings performance. While AAPL and GOOGL maintain higher stock prices, NVDA's recent earnings surprise may pose challenges. Investors should consider these factors, along with broader market conditions, when making investment decisions in the tech sector. Continuous monitoring of earnings reports and market trends will be essential for assessing future performance.

---

## References
- [Market Data Sources]
- [Earnings Reports]
- [Industry Analysis Reports]

By incorporating these enhancements, this report aims to provide a clearer, more objective, and comprehensive resource for readers interested in the stock performance of NVIDIA, Apple, and Google.


Step 5: 💾 Learning from the analysis...
   - Memory added for GOOGL
   - Memory added for NVDA
   - Memory added for AAPL


C:\Users\SyedM\AppData\Local\Temp\ipykernel_34924\417482173.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


### Question 1:

In [18]:
# Define a new research topic that requires economic data
ECONOMIC_RESEARCH_TOPIC = "Analyze Apple's (AAPL) stock performance in the context of US inflation (CPI) and unemployment."

# Instantiate the agent
agent = InvestmentResearchAgent()

# Run the full research workflow
agent.run(ECONOMIC_RESEARCH_TOPIC)

Initializing tools...
Tools initialized. Agent is ready. 🚀
Step 1: 🧠 Creating a research plan...
Plan created:
  1. get_news for AAPL
  2. process_news for AAPL
  3. get_market_data for AAPL
  4. get_earnings for AAPL
  5. get_economic_data for ['CPIAUCSL', 'UNRATE']
  6. generate_price_chart for AAPL
  7. generate_earnings_chart for AAPL
  8. generate_stock_vs_economic_chart for AAPL
  9. generate_stock_vs_economic_chart for AAPL

Step 2: 🛠️ Executing the plan (Routing)...
  Executing task: get_news...
  Executing task: process_news...
  Executing task: get_market_data...
  Executing task: get_earnings...
  Executing task: get_economic_data...
  Executing task: generate_price_chart...
   - Chart saved to: reports/images\AAPL_price_history.png
  Executing task: generate_earnings_chart...
   - Skipping earnings chart for AAPL: no surprise data.
  Executing task: generate_stock_vs_economic_chart...
   - Chart saved to: reports/images\AAPL_vs_CPIAUCSL.png
  Executing task: generate_stock_

# Analysis of Apple's (AAPL) Stock Performance in the Context of US Inflation (CPI) and Unemployment

## Executive Summary

This report provides a comprehensive analysis of Apple Inc. (AAPL) stock performance in relation to key economic indicators, specifically the Consumer Price Index (CPI) and the unemployment rate (UNRATE) in the United States. The analysis covers recent stock price movements, earnings expectations, and broader economic trends, including inflation and employment data.

### Key Findings:
- AAPL's stock has demonstrated resilience, currently trading at **$171.76**.
- The CPI has shown a consistent upward trend, indicating persistent inflationary pressures that may impact consumer spending and Apple's revenue.
- The unemployment rate remains low at **4.3%**, suggesting a stable labor market that supports consumer spending on premium products.
- Recent developments, such as the launch of Apple's M5 chip and exclusive broadcasting rights for Formula 1, present potential growth avenues that could positively influence stock performance.

## 1. Introduction

Apple Inc. (AAPL) is a leading technology company renowned for its innovative products and services. Analyzing its stock performance necessitates an understanding of external economic factors, particularly inflation and unemployment, which significantly influence consumer behavior and corporate profitability.

## 2. Stock Performance Overview

### 2.1 Recent Stock Trends

As of **October 24, 2023**, AAPL's stock closed at **$171.76**, reflecting a slight decline from previous trading days. The stock has experienced fluctuations, with a recent high of **$176.115** and a low of **$170.965** within the same week. The trading volume has varied, indicating investor interest and market activity.

### 2.2 Earnings Expectations

Looking ahead, AAPL's earnings reports are anticipated to show continued growth, with estimates suggesting earnings per share (EPS) of **$1.7924** for the upcoming quarter. Revenue expectations are robust, estimated at **$103.7 billion**, reflecting confidence in Apple's ability to maintain its market position despite economic headwinds.

## 3. Economic Context

### 3.1 Inflation (CPI)

The Consumer Price Index (CPI) has shown a consistent upward trend, reaching **323.364** in **September 2023**. This increase indicates ongoing inflationary pressures that could affect consumer purchasing power. Higher inflation may lead consumers to prioritize essential goods over premium products, potentially impacting Apple's sales.

### 3.2 Unemployment Rate

The unemployment rate has remained low, recorded at **4.3%** in **September 2023**. A stable labor market typically supports consumer spending, which is crucial for companies like Apple that rely on discretionary spending for their products. The low unemployment rate suggests that consumers may still have the financial means to invest in high-end technology.

## 4. Recent Developments and Market Sentiment

Recent news highlights several positive developments for Apple:
- The launch of the **M5 chip** showcases Apple's commitment to innovation, particularly in artificial intelligence, which could enhance product offerings and attract new customers.
- Securing exclusive broadcasting rights for **Formula 1** starting in **2026** positions Apple as a key player in the media landscape, potentially driving subscription growth and increasing brand loyalty.

Market sentiment surrounding AAPL remains positive, bolstered by its inclusion in high-performing ETFs like the **Vanguard Information Technology ETF**, which has generated an average annual return of **23.5%** over the past decade.

## 5. Comparative Analysis

To provide a more comprehensive view of AAPL's market position, it is essential to compare its performance with that of its competitors, such as Microsoft and Google, under similar economic conditions. This comparison will highlight AAPL's resilience and adaptability in the face of economic challenges.

## 6. Conclusion

In conclusion, while AAPL's stock performance is influenced by broader economic factors such as inflation and unemployment, recent developments indicate a strong potential for growth. The company's innovative products and strategic moves in media and technology position it well to navigate the challenges posed by inflationary pressures. Investors should remain optimistic about AAPL's long-term prospects, particularly as the labor market remains stable, supporting consumer spending.

## 7. Recommendations

- **Investors** should consider maintaining or increasing their positions in AAPL, given its strong fundamentals and growth potential.
- **Monitor economic indicators** such as CPI and unemployment rates to assess future stock performance and consumer behavior.
- **Stay informed** about Apple's product launches and strategic initiatives to gain insights into its market positioning and potential revenue growth.
- **Consider specific price targets** for AAPL stock, such as a buy recommendation if it dips below **$170** or a sell recommendation if it exceeds **$180**.

## 8. Limitations

This analysis relies on historical data and external economic factors that could change rapidly. Future projections are subject to uncertainty, and investors should consider these limitations when making decisions.

---

This report synthesizes the available data and provides a comprehensive analysis of AAPL's stock performance in the context of US inflation and unemployment, offering valuable insights for investors and stakeholders.


Step 5: 💾 Learning from the analysis...
   - Memory added for AAPL


C:\Users\SyedM\AppData\Local\Temp\ipykernel_34924\417482173.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


### Question 2:

In [19]:
# Define the research topic for the agent
RESEARCH_TOPIC = "Compare the recent performance and earnings of NVIDIA (NVDA), Apple (AAPL) and Microsoft (MSFT)."

# Instantiate the agent
agent = InvestmentResearchAgent()

# Run the full research workflow
agent.run(RESEARCH_TOPIC)

Initializing tools...
Tools initialized. Agent is ready. 🚀
Step 1: 🧠 Creating a research plan...
Plan created:
  1. get_news for NVDA
  2. get_news for AAPL
  3. get_news for MSFT
  4. process_news for NVDA
  5. process_news for AAPL
  6. process_news for MSFT
  7. get_market_data for NVDA
  8. get_market_data for AAPL
  9. get_market_data for MSFT
  10. get_earnings for NVDA
  11. get_earnings for AAPL
  12. get_earnings for MSFT
  13. generate_price_chart for NVDA
  14. generate_price_chart for AAPL
  15. generate_price_chart for MSFT
  16. generate_earnings_chart for NVDA
  17. generate_earnings_chart for AAPL
  18. generate_earnings_chart for MSFT
  19. generate_comparative_table for ['NVDA', 'AAPL', 'MSFT']
  20. get_economic_data for ['CPIAUCSL']
  21. generate_stock_vs_economic_chart for NVDA
  22. generate_stock_vs_economic_chart for AAPL
  23. generate_stock_vs_economic_chart for MSFT
  24. generate_comparative_price_chart for ['NVDA', 'AAPL', 'MSFT']

Step 2: 🛠️ Executing the

# Comparative Analysis of NVIDIA (NVDA), Apple (AAPL), and Microsoft (MSFT)

## Executive Summary
This report presents a detailed analysis of the recent performance and earnings of three leading technology companies: **NVIDIA (NVDA)**, **Apple (AAPL)**, and **Microsoft (MSFT)**. Each company has shown significant growth potential, particularly within the rapidly expanding artificial intelligence (AI) sector. The analysis encompasses recent market data, earnings reports, and strategic developments that underscore their competitive positions.

### Key Findings:
- **NVIDIA** maintains a dominant position in the AI chip market, showcasing substantial revenue growth and strategic partnerships.
- **Apple** continues to innovate with its new M5 chip and has secured exclusive broadcasting rights for Formula 1, enhancing its content offerings.
- **Microsoft** remains a formidable player in the tech sector, benefiting from its inclusion in high-performing ETFs and a strong focus on AI investments.

## 1. Recent Performance Overview

### 1.1 NVIDIA (NVDA)
- **Latest Close**: $183.22
- **Market Sentiment**: Positive
- **Key Developments**:
  - NVIDIA reported an impressive annual revenue of **$130 billion**, primarily driven by AI chip sales.
  - The company announced a strategic partnership with **Intel** to bolster its AI capabilities.
  - Projections indicate NVIDIA could generate **$500 billion** from AI technology by **2029**.

#### Market Data
| Date       | Open    | High    | Low     | Close   | Volume      |
|------------|---------|---------|---------|---------|-------------|
| 2023-10-24 | 43.0516 | 43.6713 | 42.6659 | 43.6373 | 401,463,000 |

### 1.2 Apple (AAPL)
- **Latest Close**: $252.29
- **Market Sentiment**: Positive
- **Key Developments**:
  - Apple launched its **M5 chip**, reinforcing its commitment to innovation.
  - The company secured exclusive U.S. broadcasting rights for **Formula 1** starting in **2026**, enhancing its content portfolio.

#### Market Data
| Date       | Open    | High    | Low     | Close   | Volume      |
|------------|---------|---------|---------|---------|-------------|
| 2023-10-24 | 171.371 | 171.985 | 169.787 | 171.758 | 43,816,600  |

### 1.3 Microsoft (MSFT)
- **Latest Close**: $513.58
- **Market Sentiment**: Positive
- **Key Developments**:
  - Microsoft is a leading holding in the **Vanguard Information Technology ETF**, which has outperformed the S&P 500.
  - The company is strategically positioned to capitalize on the ongoing AI investment boom.

#### Market Data
| Date       | Open    | High    | Low     | Close   | Volume      |
|------------|---------|---------|---------|---------|-------------|
| 2023-10-24 | 326.381 | 326.913 | 322.736 | 325.623 | 31,153,600  |

## 2. Earnings Analysis

### 2.1 NVIDIA (NVDA)
- **EPS Estimate**: $1.5242
- **EPS Actual**: $1.08 (missed estimate)
- **Revenue Estimate**: $65.41 billion
- **Revenue Actual**: $46.743 billion (missed estimate)

### 2.2 Apple (AAPL)
- **EPS Estimate**: $1.8424
- **Revenue Estimate**: $103.73 billion
- **Next Earnings Report**: Scheduled for **April 29, 2026**.

### 2.3 Microsoft (MSFT)
- **EPS Estimate**: $3.928
- **Revenue Estimate**: $82.21 billion
- **Next Earnings Report**: Scheduled for **April 28, 2026**.

## 3. Comparative Performance Metrics

| Symbol | Latest Close | Latest EPS Surprise |
|--------|---------------|---------------------|
| NVDA   | $183.22      | -0.1851             |
| AAPL   | $252.29      | N/A                 |
| MSFT   | $513.58      | N/A                 |

## 4. Comparative Analysis
### Performance Metrics
- **Revenue Growth**: NVIDIA's revenue growth is primarily driven by AI chip sales, while Apple and Microsoft are expanding their offerings and market reach.
- **Market Position**: NVIDIA leads in AI technology, Apple excels in consumer electronics and content, and Microsoft is strong in software and cloud services.

### Risks and Challenges
- **NVIDIA**: Faces competition in the AI chip market and potential supply chain disruptions.
- **Apple**: Must navigate regulatory scrutiny and market saturation in consumer electronics.
- **Microsoft**: Needs to maintain its competitive edge amid rapid technological advancements and evolving market demands.

## 5. Conclusion
NVIDIA, Apple, and Microsoft are well-positioned within the technology sector, particularly as they leverage the growth of AI and innovation. While NVIDIA has demonstrated remarkable revenue growth and strategic partnerships, Apple continues to innovate and expand its content offerings. Microsoft remains a strong contender with its significant presence in high-performing ETFs and a focus on AI investments. Investors should consider these factors, along with potential risks, when evaluating investment opportunities in these companies.

## 6. Visual Data
![Comparative Price History](reports/images/comparative_price_history.png)

## 7. References
- [NVIDIA News](https://www.fool.com/investing/2025/10/19/the-smartest-growth-stock-to-buy-with-1000-now/?source=iedfolrf0000001)
- [Apple News](https://www.fool.com/investing/2025/10/19/meet-the-only-vanguard-etf-that-has-turned-10000-i/?source=iedfolrf0000001)
- [Microsoft News](https://www.fool.com/investing/2025/10/19/1-top-stock-to-buy-to-cash-in-on-this-once-in-a-ge/?source=iedfolrf0000001)

By implementing these enhancements, the report achieves greater clarity, objectivity, and completeness, providing a robust analysis of the performance and earnings of NVIDIA, Apple, and Microsoft.


Step 5: 💾 Learning from the analysis...
   - Memory added for NVDA
   - Memory added for MSFT
   - Memory added for AAPL


C:\Users\SyedM\AppData\Local\Temp\ipykernel_34924\417482173.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")


In [20]:
# Define the research topic for the agent
RESEARCH_TOPIC = "Compare the recent performance and earnings of NVIDIA (NVDA) and Goldman Sachs (GS)."

# Instantiate the agent
agent = InvestmentResearchAgent()

# Run the full research workflow
agent.run(RESEARCH_TOPIC)

Initializing tools...
Tools initialized. Agent is ready. 🚀
Step 1: 🧠 Creating a research plan...
Plan created:
  1. get_news for NVDA
  2. get_news for GS
  3. process_news for NVDA
  4. process_news for GS
  5. get_market_data for NVDA
  6. get_market_data for GS
  7. get_earnings for NVDA
  8. get_earnings for GS
  9. generate_price_chart for NVDA
  10. generate_price_chart for GS
  11. generate_earnings_chart for NVDA
  12. generate_earnings_chart for GS
  13. generate_comparative_table for ['NVDA', 'GS']
  14. get_economic_data for ['CPIAUCSL', 'GDP']
  15. generate_stock_vs_economic_chart for NVDA
  16. generate_stock_vs_economic_chart for GS

Step 2: 🛠️ Executing the plan (Routing)...
  Executing task: get_news...
  Executing task: get_news...
  Executing task: process_news...
  Executing task: process_news...
  Executing task: get_market_data...
  Executing task: get_market_data...
  Executing task: get_earnings...
  Executing task: get_earnings...
  Executing task: generate_pri

# Comparative Analysis of NVIDIA (NVDA) and Goldman Sachs (GS)

## Executive Summary
This report presents a detailed comparative analysis of the recent performance and earnings of **NVIDIA (NVDA)** and **Goldman Sachs (GS)**. Both companies operate in rapidly evolving sectors—NVIDIA in technology and artificial intelligence (AI), and Goldman Sachs in financial services. The analysis covers their market performance, earnings surprises, strategic initiatives, and overall market sentiment, providing insights for potential investors.

## 1. Company Overview

### 1.1 NVIDIA (NVDA)
**NVIDIA** is a leading player in the AI chip market, with an impressive annual revenue of approximately **$130 billion**. The company is committed to enhancing its AI capabilities through strategic partnerships, including a recent collaboration with **Intel**. NVIDIA aims to generate **$500 billion** in AI technology by **2029**, with a strong emphasis on domestic manufacturing of AI chips.

### 1.2 Goldman Sachs (GS)
**Goldman Sachs** is a prominent investment bank that is expanding its focus on AI infrastructure financing. The firm has established a dedicated team within its global banking and markets division to capitalize on the growing demand for financing data centers related to AI. This strategic initiative reflects a positive outlook on the future of AI and its associated financial opportunities.

## 2. Recent Market Performance

### 2.1 NVIDIA Market Data
- **Latest Close**: **$183.22**
- **Recent Price Movement**: The stock has shown volatility, with a recent high of **$43.67** and a low of **$40.92** over the past week.
- **Volume**: Trading volume peaked at over **627 million shares** on **October 18, 2023**.

### 2.2 Goldman Sachs Market Data
- **Latest Close**: **$750.77**
- **Recent Price Movement**: Goldman Sachs has experienced a more stable price range, with a recent high of **$293.01** and a low of **$284.31**.
- **Volume**: The trading volume has been lower compared to NVIDIA, peaking at around **3.46 million shares**.

## 3. Earnings Performance

### 3.1 NVIDIA Earnings Data
- **Latest EPS Estimate**: **$1.52** (for the next report on **May 26, 2026**)
- **Latest EPS Actual**: **$1.08** (for the report on **November 19, 2025**)
- **EPS Surprise**: **-0.1851**, indicating that actual earnings fell short of estimates.

### 3.2 Goldman Sachs Earnings Data
- **Latest EPS Estimate**: **$13.58** (for the next report on **July 14, 2026**)
- **Latest EPS Actual**: **$12.25** (for the report on **October 14, 2025**)
- **EPS Surprise**: Not available for recent reports, but the firm has consistently aimed for strong earnings performance.

## 4. Strategic Initiatives

### 4.1 NVIDIA
- **AI Leadership**: NVIDIA continues to lead the AI chip market, with significant revenue and strategic partnerships aimed at enhancing its capabilities.
- **Domestic Manufacturing**: The company is focusing on U.S.-made technology to bolster its manufacturing capabilities.

### 4.2 Goldman Sachs
- **AI Infrastructure Financing**: Goldman Sachs is strategically positioning itself to take advantage of the booming market for AI infrastructure financing, enhancing its lending capabilities and attracting investment in AI-related projects.

## 5. Market Sentiment and Future Outlook

### 5.1 NVIDIA
The sentiment surrounding NVIDIA is largely positive, driven by its leadership in the AI sector and ambitious growth plans. However, analysts have expressed concerns about potential market downturns due to valuation issues and increased competition.

### 5.2 Goldman Sachs
Goldman Sachs also enjoys a positive sentiment, particularly with its strategic focus on AI infrastructure financing. The firm is well-positioned to capitalize on the growing demand for AI-related financial services, although it faces challenges from regulatory scrutiny and economic fluctuations.

## 6. Comparative Summary

| Metric                     | NVIDIA (NVDA) | Goldman Sachs (GS) |
|----------------------------|----------------|---------------------|
| Latest Close               | **$183.22**    | **$750.77**         |
| Latest EPS Surprise         | **-0.1851**    | **N/A**             |
| Market Focus               | **AI Technology**  | **AI Infrastructure** |
| Strategic Initiatives       | Partnerships with Intel, Domestic Manufacturing | Dedicated AI Financing Team |

## Conclusion
Both NVIDIA and Goldman Sachs are navigating their respective markets with strategic initiatives aimed at capitalizing on the growth of AI. While NVIDIA leads in technology and innovation, Goldman Sachs leverages its financial expertise to support the burgeoning AI infrastructure sector. Investors should consider the strengths and challenges of each company, including potential risks such as competition and regulatory issues, as they evaluate investment opportunities in these dynamic sectors.

## References
- Financial reports from NVIDIA and Goldman Sachs
- Market analysis from reputable financial news sources
- Industry reports on AI technology and infrastructure financing

By addressing the critiques and enhancing the clarity, objectivity, and completeness of this report, we provide a robust analysis of NVIDIA and Goldman Sachs, aiding investors in making informed decisions.


Step 5: 💾 Learning from the analysis...
   - Memory added for GS
   - Memory added for NVDA


C:\Users\SyedM\AppData\Local\Temp\ipykernel_34924\417482173.py:29: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
